In [ ]:
import pandas as pd
import numpy as np
import joblib

In [ ]:
with open('data/my_val.pkl', 'rb') as f:
    data = joblib.load(f)
    print(data.keys())
locals().update(data)
del data
np.random.seed(123)

dict_keys(['x', 'y', 'val_x', 'val_y'])


In [ ]:
test = pd.read_csv('data/test_ver03.csv', index_col='STU_ID')
idx2job = pd.read_csv('data/idx2job.csv', index_col=0)
idx2job = idx2job['job'].to_dict()
test.head()

,X1SEX,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,X1TXMTSCOR,X1TXMPROF1,X1TXMPROF2,X1TXMPROF3,X1TXMPROF4,...,X1TSCOMM,X1TSEFF,X1TSEXP,X1TSPRINC,X1TSRESP,X1SCHOOLCLI,X1COUPERTEA,X1COUPERCOU,X1COUPERPRI,birthmonth
STU_ID,,,,,,,,,,,,,,,,,,,,,
12424,1,199509.0,-0.1663,0.2511,37.7431,49.0240,0.9827,0.6242,0.2338,0.0648,...,0.01,0.17,0.16,-0.13,-0.13,-1.19,0.16,1.15,1.10,9
14300,1,199503.0,-0.6779,0.2836,29.9986,43.7132,0.8859,0.2278,0.0824,0.0468,...,1.32,-0.09,0.16,0.56,-0.33,1.00,1.07,-0.02,1.10,3
29579,1,199508.0,1.1883,0.2463,54.7405,63.0867,0.9999,0.9881,0.9742,0.5761,...,0.01,0.17,0.16,-0.13,-0.13,0.14,0.03,0.26,0.20,8
28217,1,199502.0,-2.1054,0.3637,17.2372,28.8934,0.0374,0.0174,0.0284,0.0371,...,1.32,-0.09,0.16,0.56,-0.33,1.00,1.07,-0.02,1.10,2
11561,2,199504.0,0.3538,0.2378,44.6988,54.4230,0.9979,0.8838,0.6065,0.1260,...,0.01,-0.71,-0.64,-0.24,0.22,-1.54,-0.60,-1.38,-1.11,4


- baseline modeling

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=16, n_estimators=300))])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
pipe = Pipeline([('전처리',MinMaxScaler()), 
                 ('알고리즘', RandomForestClassifier(max_depth=16, n_estimators=300))])

In [ ]:
pipe.fit(x,y)

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=16, n_estimators=300))])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, pipe.predict(val_x))

0.37103918228279387

In [ ]:
#0.38129
pipe_base = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : pipe.predict(test)})
pipe_base['X1STU30OCC2'] = pipe_base['X1STU30OCC2'].replace(idx2job)
pipe_base.to_csv('pipe_base.csv', index=False, encoding='utf-8-sig')

- one-vs-all modeling

In [ ]:
my_class = y.unique()
my_class

array([ 2, 11,  5,  0,  3,  6, 10,  8,  9, 13, 16,  1, 15,  4,  7, 12, 14,
       18, 17, 19, 20])

In [ ]:
my_models = []
for s in my_class:
    target = x.loc[y[y==s].index]
    #non_target = x.loc[y[y!=s].index]
    # undersampling코드
    non_target = x.loc[y[y!=s].index].sample(len(target))
    my_sample = pd.concat([target, non_target])
    my_sample = my_sample.sample(len(my_sample))
    my_y = pd.concat([pd.Series(1, index=y.loc[target.index].index),
                      pd.Series(0, y.loc[non_target.index].index)])
    my_y = my_y.loc[my_sample.index]

#### 모델 학습해서 my_models에 append, 1을 오바되게 잘 맞추면 점수 잘나옴
    pipe = Pipeline([('전처리',MinMaxScaler()), 
                     ('알고리즘', RandomForestClassifier(max_depth=16, n_estimators=300))])
    pipe.fit(my_sample, my_y)
####    
    my_models.append(pipe)

- Undersampling?
- Pipeline? 최적모델 탐색

In [ ]:
#21개 모델에 대한 예측 결과 생성(validation)
pred = np.zeros((len(val_x), len(my_class)))
for i, m in enumerate(my_models):
    p = m.predict_proba(val_x)[:,1]
    pred[:,i] = p
pred_class = [my_class[i] for i in pred.argmax(axis=1)]

In [ ]:
accuracy_score(val_y, pred_class)

0.36626916524701875

In [ ]:
#21개 모델에 대한 예측 결과 생성(test) : 0.37532
pred = np.zeros((len(test), len(my_class)))
for i, m in enumerate(my_models):
    p = m.predict_proba(test)[:,1]
    pred[:,i] = p
pred_class = [my_class[i] for i in pred.argmax(axis=1)]
one_vs_all = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : pred_class})
one_vs_all['X1STU30OCC2'] = one_vs_all['X1STU30OCC2'].replace(idx2job)
one_vs_all.to_csv('one_vs_all.csv', index=False, encoding='utf-8-sig')

In [ ]:
with open('data/one-vs-all.pkl', 'wb') as f:
    joblib.dump({'my_models':my_models},f)